## Built-in Tool - DuckDuckGo Search

In [2]:
from langchain_community.tools import DuckDuckGoSearchRun
from IPython.display import Markdown, display
search_tool = DuckDuckGoSearchRun()

results = search_tool.invoke('top news in india today')

display(Markdown(results))

1 hour ago · Picks for you · The Indian Express. Actor-politician Vijay's wife Sangeetha files for divorce; alleges affair with an actress. 11 hours ago. By Arun Janardanan. 6 hours ago · India News · Odisha Woman Raped Twice In 1 Day, Dies After Being Thrown Off Building: Cops · India Carries Out Successful Flight Trials Of Short-Range Air ... 2 days ago · ... News & current affairs. Join us as we explore the world and bring you the latest news from around India, the Indian Ocean Region, Indo Pacific and beyond! 3 days ago · President Trump said Indian Prime Minister Narendra Modi agreed to stop buying Russian oil and boost American purchases. By. Gavin Bade. and. Aug 31, 2025 · India fires missiles into Pakistani territory in what Islamabad calls 'act of war' ... Indian Prime Minister Narendra Modi arrives for a ceremonial reception.

In [3]:
print(search_tool.name)
print(search_tool.description)
print(search_tool.args)

duckduckgo_search
A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
{'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}


## Built-in Tool - Shell Tool

In [5]:
from langchain_community.tools import ShellTool
shell_tool = ShellTool()

results = shell_tool.invoke('dir')

print(results)

Executing command:
 dir
 Volume in drive C is Windows-SSD
 Volume Serial Number is EA76-974C

 Directory of c:\Users\mrinm\Downloads\MyTraining\GenerativeAI\RAG\CampusX_Langchain_in_depth\RAG

02/27/2026  07:29 PM    <DIR>          .
02/26/2026  11:57 PM    <DIR>          ..
02/27/2026  07:44 PM             2,967 Langchain_Tools.ipynb
02/27/2026  07:42 PM            49,504 Youtube_Chatbot_RAG.ipynb
               2 File(s)         52,471 bytes
               2 Dir(s)  902,255,386,624 bytes free



## Custom Tools

### Method1: using tool decorator class

In [6]:
from langchain_core.tools import tool

In [7]:
@tool
def multiply(a: int, b:int) -> int:
    """Multiply two numbers"""
    return a*b

In [8]:
result = multiply.invoke({"a":3, "b":5})

In [9]:
print(result)

15


### Method2: Using StructuredTool

In [11]:
from langchain_core.tools import StructuredTool
from pydantic import BaseModel, Field

In [22]:
class MultiplyInput(BaseModel):
    a: int = Field(description="The first number to add")
    b: int = Field(description="The second number to add")

In [13]:
def multiply_func(a: int, b: int) -> int:
    return a * b

In [14]:
multiply_tool = StructuredTool.from_function(
    func=multiply_func,
    name="multiply",
    description="Multiply two numbers",
    args_schema=MultiplyInput
)

In [16]:
result = multiply_tool.invoke({'a':3, 'b':3})

print(result)

9


In [17]:
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'title': 'B', 'type': 'integer'}}


### Method3: Using BaseTool Class

In [19]:
from langchain_core.tools import BaseTool
from typing import Type
from pydantic import BaseModel, Field

In [ ]:
# arg schema using pydantic
class MultiplyInput(BaseModel):
    a: int = Field(description="The first number to add")
    b: int = Field(description="The second number to add")

In [23]:
class MultiplyTool(BaseTool):
    name: str = "multiply"
    description: str = "Multiply two numbers"

    args_schema: Type[BaseModel] = MultiplyInput

    def _run(self, a: int, b: int) -> int:
        return a * b

In [24]:
multiply_tool = MultiplyTool()

In [25]:
result = multiply_tool.invoke({'a':3, 'b':3})

print(result)

9


In [26]:
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'title': 'B', 'type': 'integer'}}


## Toolkit

In [27]:
from langchain_core.tools import tool

# Custom tools
@tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b

In [28]:
class MathToolkit:
    def get_tools(self):
        return [add, multiply]

In [29]:
toolkit = MathToolkit()
tools = toolkit.get_tools()

for tool in tools:
    print(tool.name, "=>", tool.description)

add => Add two numbers
multiply => Multiply two numbers


## Tool Call by LLMs

In [30]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [31]:
# tool create
@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

In [32]:
print(multiply.invoke({'a':3, 'b':4}))

12


### Tool binding

In [39]:
llm = ChatOpenAI(model="gpt-4o", temperature=0.7)

In [40]:
llm_with_tools = llm.bind_tools([multiply])

In [41]:
llm_with_tools.invoke('Hi how are you?')

AIMessage(content="Hello! I'm here to help you with any questions or tasks you have. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 58, 'total_tokens': 81, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_64dfa806c7', 'id': 'chatcmpl-DE3yKMuRHNtaqicweyt6yG7DennYO', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019ca1fc-e5f2-7000-ad36-247010c3a76d-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 58, 'output_tokens': 23, 'total_tokens': 81, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [42]:
query = HumanMessage('can you multiply 3 with 1000')

In [43]:
messages = [query]

In [44]:
result = llm_with_tools.invoke(messages)

In [51]:
result.tool_calls[0]

{'name': 'multiply',
 'args': {'a': 3, 'b': 1000},
 'id': 'call_3UIWAlxGxkdfsMWSvK7n3DiQ',
 'type': 'tool_call'}

In [45]:
messages.append(result)

In [46]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 62, 'total_tokens': 80, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_64dfa806c7', 'id': 'chatcmpl-DE3zXTURnp7Jfwpxc762aFbMUrh0d', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019ca1fe-0b87-7d01-bc4e-016a0f3ef73c-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'call_3UIWAlxGxkdfsMWSvK7n3DiQ', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 62, 'output_tokens': 18, 'total_tokens': 80, 'input_token_details': {'audio

In [47]:
tool_result = multiply.invoke(result.tool_calls[0])

In [48]:
print(tool_result)

content='3000' name='multiply' tool_call_id='call_3UIWAlxGxkdfsMWSvK7n3DiQ'


In [49]:
messages.append(tool_result)

In [50]:
llm_with_tools.invoke(messages).content

'The product of 3 and 1000 is 3000.'

## Currency Conversion Tool

In [87]:
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, ToolMessage
import requests

@tool
def convert_currency(amount: float, base_currency: str, target_currency: str) -> float:
    """Given an amount, base currency and target currency, fetches the exchange rate and returns the converted amount."""
    url = f"https://v6.exchangerate-api.com/v6/2001318a70f721def57ea35e/pair/{base_currency}/{target_currency}"
    response = requests.get(url)
    rate = response.json()['conversion_rate']
    return amount * rate

# Bind tool to LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0)
llm_with_tools = llm.bind_tools([convert_currency])

# Start conversation
messages = [HumanMessage("Convert 100 USD to INR")]

# Step 1: LLM decides to call the tool
ai_message = llm_with_tools.invoke(messages)
messages.append(ai_message)

# Step 2: Execute the tool
tool_result = convert_currency.invoke(ai_message.tool_calls[0])
messages.append(tool_result)

# Step 3: LLM gives final answer
final = llm_with_tools.invoke(messages)
print(final.content)

100 USD is approximately 9111.13 INR.
